In [2]:
import numpy as np
import pandas as pd

# Load the preprocessed-text dataset

In [256]:
df = pd.read_csv('Hyperpartisan meta/hyper_preprocessed.csv')
#df = pd.read_csv('Fake news meta/fake_preprocessed.csv')
df.head()

,doc_id,title,mainText,hyperpartisan,title_mainText,title_preprocessed,mainText_preprocessed,title_mainText_preprocessed,mainText_verbs,mainText_nouns
0,0,Jerry Springer Just Summed Up Trump’s Debate W...,Millions of people tuned in Monday night to wa...,True,Jerry Springer Just Summed Up Trump’s Debate W...,jerry springer summed trump debate perfect tweet,million people tune monday night watch coverag...,jerry springer summed trump debate perfect twe...,"['tune', 'watch', 'go', 'come', 'report', 'bro...","['million', 'people', 'monday', 'night', 'cove..."
1,1,Clinton Campaign Charges College Students $500...,The Clintons understand the average American. ...,True,Clinton Campaign Charges College Students $500...,clinton campaign charge college students atten...,clinton understand average american know like ...,clinton campaign charge college students atten...,"['understand', 'know', 'make', 'suffer', 'lie'...","['clinton', 'broke', 'hillary', 'clinton', 'wo..."
2,2,Migrant Reveals Sick Reason He Left…’to F*ck t...,Harassment is known in Arabic as ‘taharrush’. ...,True,Migrant Reveals Sick Reason He Left…’to F*ck t...,migrant reveals sick reason woman,harassment know arabic taharrush mass sexual a...,migrant reveals sick reason woman harassment k...,"['know', 'document', 'begin', 'call', 'hide', ...","['harassment', 'arabic', 'taharrush', 'mass', ..."
3,3,Trump Said Obama Gave Iran “$150 Billion.” He ...,Democratic President Barack Obama pulled off a...,True,Trump Said Obama Gave Iran “$150 Billion.” He ...,trump said obama gave iran give nothing,democratic president barack obama pull histori...,trump said obama gave iran give nothing democr...,"['pull', 'engineer', 'freeze', 'lift', 'strang...","['president', 'barack', 'obama', 'coup', 'acco..."
4,4,Hillary Accused Trump Of Calling Climate Chang...,Democratic nominee Hillary Clinton knows her f...,True,Hillary Accused Trump Of Calling Climate Chang...,hillary accuse trump calling climate change ch...,democratic nominee hillary clinton know fact t...,hillary accuse trump calling climate change ch...,"['know', 'bust', 'call', 'rely', 'slam', 'rat'...","['nominee', 'hillary', 'clinton', 'fact', 'deb..."


In [257]:
len(df)

2234

In [258]:
#df['veracity'].value_counts()
df['hyperpartisan'].value_counts()

False    1228
True     1006
Name: hyperpartisan, dtype: int64

# Training

In [260]:
from sklearn.model_selection import train_test_split, cross_val_predict
X = df['title_mainText']
#X = df['title_mainText'] 
#X = df['mainText_nouns']
#y = df['veracity']
y = df['hyperpartisan']

# Transform Counts to Frequencies with Tf-idf, assign Model

In [261]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics, ensemble, svm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import classification_report, f1_score

text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('tfidf', TfidfTransformer()),
#                     ('clf', RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0))
#                     ('clf', SVC(C=10, kernel= 'linear', max_iter= 1000, tol=0.0001, probability=True))
                     ('clf', LogisticRegression(n_jobs=1, C=1e6)),
#                     ('clf', LinearSVC(C=1000, max_iter = 100, penalty = 'l2', loss= 'hinge' )),
])



# Cross Validation

In [262]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(text_clf, X, y, cv=10) 
score

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.92857143, 0.90625   , 0.91071429, 0.9375    , 0.92857143,
       0.93303571, 0.84753363, 0.69955157, 0.71621622, 0.72072072])

In [263]:
print (score.mean())

0.8528664995873516


In [264]:
from sklearn import metrics
scores = cross_val_score(text_clf, X, y, cv=10, scoring='f1_macro')
scores                                              

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.92726682, 0.90362038, 0.90949495, 0.93677419, 0.92774194,
       0.93204846, 0.84526531, 0.69836271, 0.71365395, 0.72069805])

In [265]:
print (scores.mean())

0.8514926758597976


# Get prediction, Compute the balanced accuracy

In [550]:
y_pred = cross_val_predict(text_clf, X, y, cv=10)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [534]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y, y_pred)

0.8439355722343465

In [535]:
from sklearn import metrics
print(metrics.classification_report(y, y_pred))

              precision    recall  f1-score   support

       False       0.86      0.86      0.86      1228
        True       0.83      0.83      0.83      1006

   micro avg       0.85      0.85      0.85      2234
   macro avg       0.84      0.84      0.84      2234
weighted avg       0.85      0.85      0.85      2234



# Grid Search

In [246]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
#              'clf__loss': ('hinge', 'squared_hinge'),
#              'tfidf__use_idf': (True, False),
              'clf__C':[1,10,100,1000],
              'clf__max_iter': [100,500,1000]}
#              'clf__penalty': ('l1', 'l2'),
#              'clf__C' : [0.01, 0.1, 1, 10, 100,1000,1e5,1e6]} 
#              'clf__kernel':['linear','rbf']}


gs_clf_svm = GridSearchCV(text_clf, parameters, cv=10)
gs_clf_svm = gs_clf_svm.fit(X,y)


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [247]:
gs_clf_svm.best_params_

{'clf__C': 1000000.0, 'vect__ngram_range': (1, 2)}

In [248]:
gs_clf_svm.best_score_

0.7760141093474426

#  feed new data to the model (Back up)

In [266]:
import numpy as np
import pandas as pd
df_new = pd.read_csv('Fake news meta/fake_preprocessed.csv')
df_new.head()

,doc_id,title,mainText,veracity,title_mainText,title_preprocessed,mainText_preprocessed,title_mainText_preprocessed,mainText_nouns,mainText_verbs
0,0,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,Results from a new study concluded that millio...,False,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,breaking study prof trump wa right every citiz...,result new study conclude million could vote p...,breaking study prof trump wa right every citiz...,"['result', 'study', 'million', 'election', 'pr...","['conclude', 'vote', 'suggest', 'cost', 'revie..."
1,1,President Trump Signed An Executive Order That...,Illegal immigrants do NOT have the same rights...,False,President Trump Signed An Executive Order That...,president trump sign executive order prevent i...,illegal immigrant right american citizen illeg...,president trump sign executive order prevent i...,"['immigrant', 'right', 'citizen', 'immigrant',...","['come', 'pay', 'seem', 'sign', 'state', 'enro..."
2,2,New Document Reveals Lynch Instructed Comey to...,"Ah, the infamous tarmac meeting. You know…the...",False,New Document Reveals Lynch Instructed Comey to...,new document reveals lynch instructed comey hi...,ah infamous tarmac meeting one former attorney...,new document reveals lynch instructed comey hi...,"['ah', 'tarmac', 'meeting', 'attorney', 'gener...","['meet', 'suspect', 'go', 'get', 'agree', 'rev..."
3,3,Newly Leaked Report Shows Hillary DID NOT Win ...,President Trump won the election fair and squa...,False,Newly Leaked Report Shows Hillary DID NOT Win ...,newly leak report show hillary win new dems pa...,president trump win election fair square evide...,newly leak report show hillary win new dems pa...,"['president', 'trump', 'election', 'fair', 'sq...","['pile', 'suggest', 'appear', 'give', 'report'..."
4,4,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,Katie Couric has come out publicly and corrobo...,False,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,katie couric speaks canceling clinton intervie...,katie couric come publicly corroborate story f...,katie couric speaks canceling clinton intervie...,"['katie', 'couric', 'story', 'sound', 'tech', ...","['come', 'corroborate', 'hear', 'refuse', 'rem..."


In [267]:
len(df_new)

2268

In [248]:
text_clf.fit(X, y) 
#hyper_prediction = text_clf.predict_proba(df_new['mainText'])[:,1]
hyper_prediction = text_clf.predict(df_new['mainText_preprocessed'])
hyperpartisan = (hyper_prediction)
result = pd.DataFrame({'doc_id': df_new['doc_id'], 'hyperpartisan': hyperpartisan})
complete= pd.merge(df_new,result,how='inner',on=['doc_id'])


/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [249]:
complete

,doc_id,title,mainText,veracity,mainText_preprocessed,hyperpartisan
0,0,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,Results from a new study concluded that millio...,False,result new study conclude million could vote p...,0.754513
1,1,President Trump Signed An Executive Order That...,Illegal immigrants do NOT have the same rights...,False,illegal immigrant right american citizen illeg...,0.809571
2,2,New Document Reveals Lynch Instructed Comey to...,"Ah, the infamous tarmac meeting. You know…the...",False,ah infamous tarmac meeting one former attorney...,0.720724
3,3,Newly Leaked Report Shows Hillary DID NOT Win ...,President Trump won the election fair and squa...,False,president trump win election fair square evide...,0.115309
4,4,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,Katie Couric has come out publicly and corrobo...,False,katie couric come publicly corroborate story f...,0.479340
5,5,SHOCK CLAIM: Hillary To Be Indicted On Crimina...,Top investigators in the Department of Justice...,False,top investigator department justice believe am...,0.465791
6,6,BOOM: PRESIDENT TRUMP SENT MICHELLE OBAMA A BI...,Michelle Obama will go down in history as the ...,False,michelle obama go history first lady spend mon...,0.986086
7,7,Obama’s Birth Brother Breaks Silence – Reveals...,Rumors have been spreading for years that Bara...,False,rumor spread year barack obama actually bear k...,0.203511
8,8,Hillary Clinton Busted in the Middle of Huge P...,An NBC news report claims that Hillary Clinton...,False,nbc news report claim hillary clinton secretar...,0.561630
9,9,Breaking: Sharia Law is Gone--The House Made a...,Former president Barack Obama and his Democrat...,False,former president barack obama democrat want u ...,0.695331


In [110]:
complete.to_csv("test2.csv",index=False)

# k-fold Cross Validation (Oversampling)

In [54]:
from random import randrange
# Split a dataset into k folds
def cross_validation_split(dataset, folds=3):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / folds)
    for i in range(folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [55]:
test=df['doc_id'].values.tolist()


In [56]:
# test cross validation split
folds = cross_validation_split(test, 3)
a = folds[0]
b = folds[1]
c = folds[2]

In [57]:
df1 = pd.DataFrame({'doc_id':a})
df1= pd.merge(df1,df,how='inner',on=['doc_id'])
df2 = pd.DataFrame({'doc_id':b})
df2= pd.merge(df2,df,how='inner',on=['doc_id'])
df3 = pd.DataFrame({'doc_id':c})
df3= pd.merge(df3,df,how='inner',on=['doc_id'])


## First cross validation

In [74]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score


smt = SMOTE(random_state=42)
clf = LinearSVC()

pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('smt', smt),
                     ('clf', clf),])

frames = [df1, df2]
df4 = df3.set_index('doc_id')
result1 = pd.concat(frames, ignore_index=True)
result=result1.set_index('doc_id')

X_train = result['mainText_preprocessed']
X_test = df4['mainText_preprocessed']
#y_train = result['hyperpartisan']
#y_test = df4['hyperpartisan']
y_train = result['veracity']
y_test = df4['veracity']

pipeline.fit(X_train, y_train) 

y_hat = pipeline.predict(X_test)
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

       False       0.56      0.38      0.46        94
        True       0.87      0.93      0.90       419

   micro avg       0.83      0.83      0.83       513
   macro avg       0.72      0.66      0.68       513
weighted avg       0.81      0.83      0.82       513



In [75]:
#score1 = accuracy_score(y_test, y_hat)
score1 = f1_score(y_test, y_hat, average='macro')  
score1

0.6783089307589103

In [ ]:
score1 = accuracy_score(y_test, y_hat)

## Second cross validation

In [77]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import classification_report

smt = SMOTE(random_state=42)
clf = LinearSVC()

pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('smt', smt),
                     ('clf', clf),])

frames = [df1, df3]
df4 = df2.set_index('doc_id')
result1 = pd.concat(frames, ignore_index=True)
result=result1.set_index('doc_id')

X_train = result['mainText_preprocessed']
X_test = df4['mainText_preprocessed']
#y_train = result['hyperpartisan']
#y_test = df4['hyperpartisan']
y_train = result['veracity']
y_test = df4['veracity']

pipeline.fit(X_train, y_train) 

y_hat = pipeline.predict(X_test)
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

       False       0.60      0.55      0.57       104
        True       0.89      0.91      0.90       409

   micro avg       0.83      0.83      0.83       513
   macro avg       0.74      0.73      0.74       513
weighted avg       0.83      0.83      0.83       513



In [78]:
#score2 = accuracy_score(y_test, y_hat)
score2 = f1_score(y_test, y_hat, average='macro')  
score2

0.735041592484794

## Third cross validation

In [79]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

smt = SMOTE(random_state=42)
clf = LinearSVC()

pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('smt', smt),
                     ('clf', clf),])

frames = [df2, df3]
df4 = df1.set_index('doc_id')
result1 = pd.concat(frames, ignore_index=True)
result=result1.set_index('doc_id')

X_train = result['mainText_preprocessed']
X_test = df4['mainText_preprocessed']
#y_train = result['hyperpartisan']
#y_test = df4['hyperpartisan']
y_train = result['veracity']
y_test = df4['veracity']

pipeline.fit(X_train, y_train) 

y_hat = pipeline.predict(X_test)
print(classification_report(y_test, y_hat))


              precision    recall  f1-score   support

       False       0.61      0.45      0.52        93
        True       0.89      0.94      0.91       420

   micro avg       0.85      0.85      0.85       513
   macro avg       0.75      0.69      0.71       513
weighted avg       0.84      0.85      0.84       513



In [80]:
#score3 = accuracy_score(y_test, y_hat)
score3 = f1_score(y_test, y_hat, average='macro')  
score3

0.7141203703703703

In [81]:
average = []
average.append(score1)
average.append(score2)
average.append(score3)
np.array(average).mean() 


0.7091569645380248